Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

Inspect your folder structure. What do you notice?

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [35]:
department = spark.read.csv("dept_copy.csv", sep=",", header=True, inferSchema=True)

In [36]:
department.columns

['dept_division', 'dept_name', 'standardized_dept_name', 'dept_subject_to_SLA']

In [2]:
source = spark.read.csv("source_copy.csv", sep=",", header=True, inferSchema=True)

In [33]:
source

['source_id', 'source_username']

In [4]:
case = spark.read.csv("case_copy.csv", sep=",", header=True, inferSchema=True)

In [34]:
case.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'council_distrtict']

In [5]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
df_source_jason = source.write.json("sources_json", mode="overwrite")

In [7]:
df_source_csv = source.write.csv("sources_csv", mode="overwrite")

In [8]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [9]:
department.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [10]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [11]:
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [12]:
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [13]:
fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

In [14]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

How old is the latest (in terms of days past SLA) currently open issue? 

How long has the oldest (in terms of days since opened) currently opened issue been open?

How many Stray Animal cases are there?

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

Convert the council_district column to a string column.

Extract the year from the case_closed_date column.
Convert num_days_late from days to hours in new columns num_hours_late.

Join the case data with the source and department data.

Are there any cases that do not have a request source?

What are the top 10 service request types in terms of number of requests?

What are the top 10 service request types in terms of average days late?
Does number of days late depend on department?
How do number of days late depend on department and request type?

In [15]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [16]:
case.filter(case.case_closed== False).select(max(case.SLA_days)).show()

+-------------+
|max(SLA_days)|
+-------------+
|   1419.00191|
+-------------+



In [19]:
case.select(
            datediff(
                      case.filter(case.case_closed==False).select(min(case.case_opened_date)),
                      case.select(max(case.case_closed_date)
                    )
          ).show()

TypeError: 'Column' object is not callable

In [53]:
import pandas as pd

#How long has the oldest (in terms of days since opened) currently opened issue been open?
start = case.filter(case.case_closed==False).select(min(case.case_opened_date).alias('date1'))
end = case.select(max(case.case_closed_date).alias('date2'))


print(start.show())
print(end.show())

datediff(lit('2017-01-01'),lit('2018-08-08'))
#pd.todatetime

+-------------------+
|              date1|
+-------------------+
|2017-01-01 13:48:00|
+-------------------+

None
+-------------------+
|              date2|
+-------------------+
|2018-08-08 10:38:00|
+-------------------+

None


Column<b'datediff(2017-01-01, 2018-08-08)'>

In [55]:
pd.to_datetime('2017-01-01') - pd.to_datetime('2018-08-08')

Timedelta('-584 days +00:00:00')

In [42]:
# How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
case.filter(case.dept_division=='Field Operations').filter(case.service_request_type!="Officer Standby").count()

113902

In [24]:
# How many Stray Animal cases are there?
case.select(case.service_request_type=='Stray Animal').count()

841704

Convert the council_district column to a string column.

In [25]:
case = case.withColumn('council_distrtict',case.council_district.cast("string"))

Extract the year from the case_closed_date column. 

Convert num_days_late from days to hours in new columns num_hours_late.

In [26]:
case.select(case.case_closed_date[1:4].alias('year')).show(5)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 5 rows



In [27]:
case.withColumn('num_days_late', expr('num_days_late * 24 AS num_hours_late')).show(5)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------------+
|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|council_distrtict|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false|     -23964.2102784|       true|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|                5|
|1014127333|2018-01-01 00:46

In [37]:
df = (
        source
        .join(case,"source_id","left")
        .join(department,"dept_division","left")
      )

In [40]:
df.show(5)

+----------------+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+----------+-----------+--------------------+----------------+-----------------+--------------------+----------------------+-------------------+
|   dept_division|source_id| source_username|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|service_request_type|  SLA_days|case_status|     request_address|council_district|council_distrtict|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+----------+-----------+--------------------+----------------+-----------------+--------------------+----------------------+-------------------+
|Waste Collection|   136202|Mic

In [64]:
df.groupby(df.request_address).count().orderBy('count').show(5)

+--------------------+-----+
|     request_address|count|
+--------------------+-----+
|1300  MARTIN LUTH...|    1|
|6458  VILLAGE PAR...|    1|
|FLOWER MEADOW and...|    1|
|6234  WELLES BROO...|    1|
|9106  WUTHERING H...|    1|
+--------------------+-----+
only showing top 5 rows



What are the top 10 service request types in terms of number of requests?

What are the top 10 service request types in terms of average days late? Does number of days late depend on department? 

How do number of days late depend on department and request type?

In [73]:
df.groupby(df.service_request_type).count().orderBy('count',ascending=False).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



What are the top 10 service request types in terms of average days late

In [89]:
df.groupby(df.service_request_type).agg(avg('num_days_late')).orderBy('avg(num_days_late)',ascending=False).show(10,truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420932|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.9285161247979 |
|Donation Container Enforcement        |131.75610506358709|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.87050230311695 |
+--------------------------------------+------------------+
only showing top 10 rows



In [94]:
df.groupby(df.dept_name,df.service_request_type).agg(avg('num_days_late')).orderBy('avg(num_days_late)',ascending=False).show(truncate=False)

+-------------------------+--------------------------------------+------------------+
|dept_name                |service_request_type                  |avg(num_days_late)|
+-------------------------+--------------------------------------+------------------+
|Code Enforcement Services|Zoning: Junk Yards                    |175.95636210420932|
|Code Enforcement Services|Labeling for Used Mattress            |162.43032902285717|
|Code Enforcement Services|Record Keeping of Used Mattresses     |153.99724039428568|
|Code Enforcement Services|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Code Enforcement Services|Storage of Used Mattress              |142.11255641500003|
|null                     |Zoning: Recycle Yard                  |135.9285161247979 |
|Code Enforcement Services|Donation Container Enforcement        |131.75610506358709|
|Code Enforcement Services|License Requied Used Mattress Sales   |128.79828704142858|
|Trans & Cap Improvements |Traffic Signal Graffiti    